In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
database_path = '../chartsite/db.sqlite3'
conn = sqlite3.connect(database_path)

In [3]:
cursor = conn.execute("select * from dashboard_report_counts")
report_counts = cursor.fetchall()
report_counts[:5]

[(0, '2020-03-01', 'РК', 51, 11501, 166, 0, '2020-03-01'),
 (1, '2020-03-01', 'ВКЛ', 132, 5568, 123, 0, '2020-03-01'),
 (2, '2020-03-01', 'НКЛ', 168, 6874, 10, 0, '2020-03-01'),
 (3, '2020-03-01', 'ОВ', 191, 3209, 26, 0, '2020-03-01'),
 (4, '2020-03-01', 'Г', 85, 1584, 145, 0, '2020-03-01')]

In [4]:
pd_data = pd.DataFrame(report_counts, columns = ['id', 'load_date', 'product_type', 
                                       'in_default', 'healthy', 'will_default', 'deleted', 'report_date'])
pd_data.head()

,id,load_date,product_type,in_default,healthy,will_default,deleted,report_date
0,0,2020-03-01,РК,51,11501,166,0,2020-03-01
1,1,2020-03-01,ВКЛ,132,5568,123,0,2020-03-01
2,2,2020-03-01,НКЛ,168,6874,10,0,2020-03-01
3,3,2020-03-01,ОВ,191,3209,26,0,2020-03-01
4,4,2020-03-01,Г,85,1584,145,0,2020-03-01


In [5]:
grouped_healhty = pd_data.groupby(by=['report_date', 'product_type'])[['healthy']].sum()
healhty_dynamics = grouped_healhty.unstack()
healhty_dynamics.columns = healhty_dynamics.columns.droplevel()
healhty_dynamics.head(5)

product_type,ВКЛ,Г,НКЛ,ОВ,РК
report_date,,,,,
2016-02-01,10738,13362,14185,14723,5875
2016-03-01,3835,19537,7313,14877,14605
2016-04-01,10206,4804,14520,19345,16145
2016-05-01,13745,5185,17026,11582,1536
2016-06-01,10795,7542,11096,14073,14018


In [56]:
pd_data['total'] = pd_data['in_default']+pd_data['healthy']
grouped_data = pd_data.groupby(by=['report_date', 'product_type'])[['in_default', 'total']].sum()
grouped_data['def_share'] = grouped_data['in_default']/grouped_data['total']
grouped_data.head(6)

in_default  total  def_share
report_date product_type                              
2016-02-01  ВКЛ                  191  10929   0.017476
            Г                    136  13498   0.010076
            НКЛ                   44  14229   0.003092
            ОВ                    72  14795   0.004867
            РК                    64   5939   0.010776
2016-03-01  ВКЛ                   84   3919   0.021434

In [19]:
from math import cos, pi, sin
from bokeh.models import Arc, Circle, ColumnDataSource, Plot, Range1d, Ray, Text





def data(value):
    return dict(value=value, units="data")

def riskmeter(risk):
    xdr = Range1d(start=-1.25, end=1.25)
    ydr = Range1d(start=-1.25, end=1.25)
    plot = Plot(x_range=xdr, y_range=ydr, plot_width=600, plot_height=300)
    radius = 100

    ##Зоны монометра
    # Зелёная зона
    glyph = Arc(x=0, y=0, radius=radius, start_angle=pi, end_angle=pi * 2 / 3,
                direction="clock", line_color='green',line_width=40)
    plot.add_glyph(glyph)
    # Жёлтая зона
    glyph = Arc(x=0, y=0, radius=radius, start_angle=pi * 2 / 3, end_angle=pi * 1 / 3, direction="clock",
                line_color='yellow', line_width=40)
    plot.add_glyph(glyph)
    # Красная зона
    glyph = Arc(x=0, y=0, radius=radius, start_angle=pi * 1 / 3, end_angle=0, direction="clock", line_color='red',
                line_width=40)
    plot.add_glyph(glyph)

    angle = (1-risk)*pi
    plot.add_glyph(Ray(x=0, y=0, length=data(0.75), angle=angle, line_color="black", line_width=3))
    return plot



In [45]:
import numpy as np

from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid
from bokeh.models.glyphs import Arc
from bokeh.io import curdoc, show

N = 9
x = np.linspace(-2, 2, N)
y = x**2
r = x/15.0+0.3

source = ColumnDataSource(dict(x=x, y=y, r=r))

plot = Plot(
    title=None, plot_width=300, plot_height=300,
    min_border=0, toolbar_location=None)

glyph = Arc(x=0, y=0, radius=0.5, start_angle=0.6, end_angle=4.1, line_color="#beaed4", line_width=3)
plot.add_glyph(source, glyph)

xaxis = LinearAxis()
plot.add_layout(xaxis, 'below')

yaxis = LinearAxis()
plot.add_layout(yaxis, 'left')

plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

curdoc().add_root(plot)

show(plot)

In [41]:
show(plot)

In [21]:
t = riskmeter(0.2)

In [22]:
show(t)

In [6]:
from bokeh.plotting import figure, show

In [12]:
plot = figure()
colors = ['red', 'green', 'blue', 'yellow','black']

In [16]:
plot = figure(plot_width=800, plot_height=300, title="Healthy",)
for i in range(len(healhty_dynamics.columns)):
        ctype=healhty_dynamics.columns[i]
        col = colors[i]
        _y = healhty_dynamics[ctype].values
        _x = [v for v in healhty_dynamics.index.values]
        plot.line(y = _y, x=healhty_dynamics.index.values, legend_label=ctype, color=col)
show(plot)

In [14]:
healhty_dynamics.index.values

array(['2016-02-01', '2016-03-01', '2016-04-01', '2016-05-01',
       '2016-06-01', '2016-07-01', '2016-08-01', '2016-09-01',
       '2016-10-01', '2016-11-01', '2016-12-01', '2017-01-01',
       '2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01',
       '2017-06-01', '2017-07-01', '2017-08-01', '2017-09-01',
       '2017-10-01', '2017-11-01', '2017-12-01', '2018-01-01',
       '2018-02-01', '2018-03-01', '2018-04-01', '2018-05-01',
       '2018-06-01', '2018-07-01', '2018-08-01', '2018-09-01',
       '2018-10-01', '2018-11-01', '2018-12-01', '2019-01-01',
       '2019-02-01', '2019-03-01', '2019-04-01', '2019-05-01',
       '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01',
       '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01',
       '2020-02-01', '2020-03-01'], dtype=object)

In [ ]:
p.multi_line()